In [45]:
import re
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
accepted_filepath = "accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv"
rejected_filepath = "rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv"

In [3]:
accepted_df = pd.read_csv(accepted_filepath, error_bad_lines=False,low_memory=False)
rejected_df = pd.read_csv(rejected_filepath, error_bad_lines=False,low_memory=False)

In [46]:
list(accepted_df)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 '

In [20]:
rejected_df.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [31]:
list(rejected_df)

['Amount Requested',
 'Application Date',
 'Loan Title',
 'Risk_Score',
 'Debt-To-Income Ratio',
 'Zip Code',
 'State',
 'Employment Length',
 'Policy Code']

In [32]:
# select Needed columns
clean_accept_df = accepted_df[[
    'loan_amnt',
    'issue_d',
    'fico_range_high',
    'dti',
    'zip_code',
    'addr_state',
    'emp_length',
    'policy_code'
]]

In [35]:
clean_reject_df = rejected_df[['Amount Requested',
 'Application Date',
 'Risk_Score',
 'Debt-To-Income Ratio',
 'Zip Code',
 'State',
 'Employment Length',
 'Policy Code']]

In [37]:
clean_accept_df.shape

(2260701, 8)

In [39]:
clean_reject_df.shape

(27648741, 8)

In [41]:
# adding a Classifications to the data
clean_accept_df['Loan Status'] = "Accepted"

C:\Users\pamponsah\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [42]:
clean_reject_df.head()

,loan_amnt,issue_d,fico_range_high,dti,zip_code,addr_state,emp_length,policy_code,Loan Status
0,3600.0,Dec-2015,679.0,5.91,190xx,PA,10+ years,1.0,Accepted
1,24700.0,Dec-2015,719.0,16.06,577xx,SD,10+ years,1.0,Accepted
2,20000.0,Dec-2015,699.0,10.78,605xx,IL,10+ years,1.0,Accepted
3,35000.0,Dec-2015,789.0,17.06,076xx,NJ,10+ years,1.0,Accepted
4,10400.0,Dec-2015,699.0,25.37,174xx,PA,3 years,1.0,Accepted


In [43]:
# adding a Classifications to the data
clean_reject_df['Loan Status'] = "Rejected"

In [44]:
clean_reject_df.head()

,Amount Requested,Application Date,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code,Loan Status
0,1000.0,2007-05-26,693.0,10%,481xx,NM,4 years,0.0,Rejected
1,1000.0,2007-05-26,703.0,10%,010xx,MA,< 1 year,0.0,Rejected
2,11000.0,2007-05-27,715.0,10%,212xx,MD,1 year,0.0,Rejected
3,6000.0,2007-05-27,698.0,38.64%,017xx,MA,< 1 year,0.0,Rejected
4,1500.0,2007-05-27,509.0,9.43%,209xx,MD,< 1 year,0.0,Rejected
